In [17]:
import pandas as pd
import numpy as np
from wrangle import *
import matplotlib.pyplot as plt

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings("ignore")


### Exercises

Do your work for this exercise in a jupyter notebook named modeling within the regression-exercises repo. Add, commit, and push your work.



1. Select a dataset with a continuous target variable. 
2. Be sure your data is prepared (no missing values, numeric datatypes) and split into samples. 
3. Work through **all** of the steps outlined in the lesson, from setting the baseline to selected a model and evaluating the final model on your test data.

In [18]:
prepped_array=prep_zillow_2017(k=1.25)


Number of observations removed: 18684
This is our percent change after removing all the outliers and then the nulls:
 -33.33%
mean kurt:
-0.3094772441000198


None

In [19]:

cols=set(prepped_array[0].columns.to_list())
colsdropa={'id','yearbuilt', 'bathdividesbed',
 'beddividestaxval',
 'bathdividestaxval',
 'areadividestaxval',
 'bedbathbeyonddividestaxval',
 'beddividesarea',
 'bathdividesarea',
 'bathplusbathdividesarea',
 'bathplusbathdividesarea_dividestavval','logerror'}

minwithdecade=cols-colsdropa

colsdropb=colsdropa-{'yearbuilt'}
colsdropb.add('decade')
minwithyearsbuilt=cols-colsdropb

print(f'''minwithdecade:\n{minwithdecade}\nminwithyearsbuilt\n{minwithyearsbuilt} ''')

minwithdecade:
{'bathroomcnt', 'bedroomcnt', 'fips', 'area', 'decade'}
minwithyearsbuilt
{'yearbuilt', 'bathroomcnt', 'bedroomcnt', 'fips', 'area'} 


In [20]:
X_train.head()
# y_train.head()

NameError: name 'X_train' is not defined

In [ ]:

# make a list to store model rmse in
rmse_list=[]
model_name_list=[]

# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

# 1. Predict taxvaluedollarcnt_pred_mean
taxvaluedollarcnt_pred_mean = y_train['taxvaluedollarcnt'].mean()
y_train['taxvaluedollarcnt_pred_mean'] = taxvaluedollarcnt_pred_mean
y_validate['taxvaluedollarcnt_pred_mean'] = taxvaluedollarcnt_pred_mean

# 2. compute taxvaluedollarcnt_pred_median
taxvaluedollarcnt_pred_median = y_train['taxvaluedollarcnt'].median()
y_train['taxvaluedollarcnt_pred_median'] = taxvaluedollarcnt_pred_median
y_validate['taxvaluedollarcnt_pred_median'] = taxvaluedollarcnt_pred_median

# 3. RMSE of taxvaluedollarcnt_pred_mean
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_mean)**(1/2)
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_mean)**(1/2)

print(f'''RMSE using Mean\nTrain/In-Sample: {rmse_train:.4g}
      \nValidate/Out-of-Sample: \n {rmse_validate:.4g}''')
rmse_list.append([rmse_train,rmse_train])
model_name_list.append('RMSE using Mean')

In [ ]:
# 4. RMSE of taxvaluedollarcnt_pred_median
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_median)**(1/2)
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_median)**(1/2)
print(f'''RMSE using Median\nTrain/In-Sample: {rmse_train:.4g}
      \nValidate/Out-of-Sample: \n {rmse_validate:.4g}''')

rmse_list.append([rmse_train,rmse_train])
model_name_list.append('RMSE using Median')

In [ ]:
# 1. Predict taxvaluedollarcnt_pred_mode
taxvaluedollarcnt_pred_mode = y_train['taxvaluedollarcnt'].mode()[0]
y_train['taxvaluedollarcnt_pred_mode'] = taxvaluedollarcnt_pred_mode
y_validate['taxvaluedollarcnt_pred_mode'] = taxvaluedollarcnt_pred_mode


# 2. RMSE of taxvaluedollarcnt_pred_mode
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_mode)**(1/2)
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_mode)**(1/2)

print(f'''RMSE using Mode\nTrain/In-Sample: {rmse_train:.2g}
      \nValidate/Out-of-Sample: \n {rmse_validate:.2g}''')
rmse_list.append([rmse_train,rmse_train])
model_name_list.append('RMSE using Mode')

In [ ]:
y_train

In [ ]:
# plot to visualize actual vs predicted. 
# plt.hist(y_train.taxvaluedollarcnt, color='blue', alpha=.5, label="Actual Tax Values")
# plt.hist(y_train.taxvaluedollarcnt_pred_mean, bins=1, color='red', alpha=.5, rwidth=1e4, label="Predicted Tax Values - Mean")
# plt.hist(y_train.taxvaluedollarcnt_pred_median, bins=1, color='orange', alpha=.5, rwidth=1e5, label="Predicted Tax Values - Median")
# plt.xlabel("Tax Value (Single Family Properties)")
# plt.ylabel("Number of Single Family Properties")
# plt.ylim(top=5000)
# plt.legend()
# plt.show()

In [ ]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train.taxvaluedollarcnt)

# predict train
y_train['taxvaluedollarcnt_pred_lm'] = lm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_lm)**(1/2)

# predict validate
y_validate['taxvaluedollarcnt_pred_lm'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lm)**(1/2)


print(f'''RMSE for OLM \nTrain/In-Sample: {rmse_train:.4g}
      \nValidate/Out-of-Sample: \n {rmse_validate:.4g}''')
rmse_list.append([rmse_train,rmse_train])
model_name_list.append('RMSE for OLM ')

In [ ]:



# create the model object
lars = LassoLars(alpha=1.0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(X_train, y_train.taxvaluedollarcnt)

# predict train
y_train['taxvaluedollarcnt_pred_lars'] = lars.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_lars)**(1/2)

# predict validate
y_validate['taxvaluedollarcnt_pred_lars'] = lars.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lars)**(1/2)


print(f'''RMSE for Lasso + Lars\nTraining/In-Sample: :  {rmse_train:.4g}
 \nValidate/Out-of-Sample: \n {rmse_validate:.4g}''')

rmse_list.append([rmse_train,rmse_train])
model_name_list.append('RMSE for Lasso + Lars ')

In [ ]:


# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train, y_train.taxvaluedollarcnt)

# predict train
y_train['taxvaluedollarcnt_pred_glm'] = glm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_glm)**(1/2)

# predict validate
y_validate['taxvaluedollarcnt_pred_glm'] = glm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_glm)**(1/2)


print(f'''RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample:  {rmse_train:.4g}
      "\nValidate/Out-of-Sample: \n {rmse_validate:.4g}''')
rmse_list.append([rmse_train,rmse_train])

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate)
X_test_degree2 = pf.transform(X_test)


In [ ]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train.taxvaluedollarcnt)

# predict train
y_train['taxvaluedollarcnt_pred_lm2'] = lm2.predict(X_train_degree2)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.taxvaluedollarcnt, y_train.taxvaluedollarcnt_pred_lm2)**(1/2)

# predict validate
y_validate['taxvaluedollarcnt_pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lm2)**(1/2)

print(f'''RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: :  {rmse_train:.4g}
 \nValidate/Out-of-Sample: \n {rmse_validate:.4g}''')

rmse_list.append([rmse_train,rmse_train])
model_name_list.append('RMSE for Polynomial Model')



In [ ]:
modelsresmedict=dict(zip(model_name_list,rmse_list))
rmseDF=pd.DataFrame(data=modelsresmedict,index=['Train','Validate'])
rmseDF=rmseDF.T
rmseDF['Diff']=rmseDF.Train-rmseDF.Validate
rmseDF

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_mean, alpha=.5, color="purple", label='_nolegend_')
plt.annotate("Baseline: Predict Using Mean", (16, 9.5))
plt.plot(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt, alpha=.5, color="blue", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5+3, 3.5+3), rotation=15.5)

plt.scatter(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lm, 
            alpha=.5, color="red", s=100, label="Model: LinearRegression")
plt.scatter(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_glm, 
            alpha=.5, color="yellow", s=100, label="Model: TweedieRegressor")
plt.scatter(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lm2, 
            alpha=.5, color="green", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual Tax Value")
plt.ylabel("Predicted Tax Value")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()





In [ ]:

# y_validate.head()
plt.figure(figsize=(16,8))
plt.axhline(label="No Error")
plt.scatter(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lm-y_validate.taxvaluedollarcnt, 
            alpha=.5, color="red", s=100, label="Model: LinearRegression")
plt.scatter(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_glm-y_validate.taxvaluedollarcnt, 
            alpha=.5, color="yellow", s=100, label="Model: TweedieRegressor")
plt.scatter(y_validate.taxvaluedollarcnt, y_validate.taxvaluedollarcnt_pred_lm2-y_validate.taxvaluedollarcnt, 
            alpha=.5, color="green", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual Tax Value")
plt.ylabel("Residual/Error: Predicted Grade - Actual Grade")
plt.title("Do the size of errors change as the actual value changes?")
plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()



In [ ]:
# plot to visualize actual vs predicted. 
plt.figure(figsize=(16,8))

# plt.hist(y_validate.taxvaluedollarcnt_pred_lm, color='red', alpha=.5, label="Model: LinearRegression")

# plt.hist(y_validate.taxvaluedollarcnt_pred_lm2, color='green', alpha=.5, label="Model 2nd degree Polynomial")
plt.hist(y_validate.taxvaluedollarcnt, color='white', alpha=.5, label="Actual Tax Values")
# plt.hist(y_validate.taxvaluedollarcnt_pred_glm, color='blue', alpha=.5, label="Model: TweedieRegressor")
plt.xlabel("Taxes (taxvaluedollarcnt)")
plt.ylabel("Number Single Family Residences")
plt.title("Comparing the Distribution of Actual Tax Values to Distributions of Predicted Tax Values for the Top Models")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(16,8))

# plt.hist(y_validate.taxvaluedollarcnt_pred_lm, color='red', alpha=.5, label="Model: LinearRegression")

plt.hist(y_validate.taxvaluedollarcnt_pred_lm2, color='green', alpha=.5, label="Model 2nd degree Polynomial")
# plt.hist(y_validate.taxvaluedollarcnt, color='white', alpha=.5, label="Actual Tax Values")
# plt.hist(y_validate.taxvaluedollarcnt_pred_glm, color='blue', alpha=.5, label="Model: TweedieRegressor")
plt.xlabel("Taxes (taxvaluedollarcnt)")
plt.ylabel("Number Single Family Residences")
plt.title("Comparing the Distribution of Actual Tax Values to Distributions of Predicted Tax Values for the Top Models")
plt.legend()
plt.show()



In [ ]:

# predict on test
y_test['taxvaluedollarcnt_pred_lm2'] = lm2.predict(X_test_degree2)


# evaluate: rmse
rmse_test = mean_squared_error(y_test.taxvaluedollarcnt, (y_test.taxvaluedollarcnt_pred_lm2)**(1/2))


print(f"RMSE for Polynomial Model, degrees=2\nTest/Out-of-Sample Performance:\n{rmse_test:.2g}")





In [ ]:


y_test['taxvaluedollarcnt_pred_glm'] = glm.predict(X_test)



# evaluate: rmse
rmse_test = mean_squared_error(y_test.taxvaluedollarcnt, (y_test.taxvaluedollarcnt_pred_glm)**(1/2))


print(f"RMSE for Polynomial Model, degrees=2\nTest/Out-of-Sample Performance:\n{rmse_test:.2g}")





